# Geneformer Multi-Task Cell Classifier Tutorial

This tutorial demonstrates how to use the Geneformer Multi-Task Cell Classifier and optimizatize hyperparameter for fine-tuning

## 1. Installation and Imports

First import the necessary modules.

In [3]:
from geneformer import MTLClassifier

## 2. Set up Paths and Parameters

Now, let's set up the necessary paths and parameters for our classifier. We'll also define our task columns, which are specific columns from our dataset that represent the classification tasks we want to train the model on.

In [ ]:
# Define paths
pretrained_path = "/path/to/pretrained/Geneformer/model" 
# input data is tokenized rank value encodings generated by Geneformer tokenizer (see tokenizing_scRNAseq_data.ipynb)
train_path = "/path/to/train/data.dataset"
val_path = "/path/to/val/data.dataset"
test_path = "/path/to/test/data.dataset"
results_dir = "/path/to/results/directory"
model_save_path = "/path/to/model/save/path"
tensorboard_log_dir = "/path/to/tensorboard/log/dir"

# Define tasks and hyperparameters
# task_columns should be a list of column names from your dataset
# Each column represents a specific classification task (e.g. cell type, disease state)
task_columns = ["cell_type", "disease_state"]  # Example task columns

hyperparameters = {
    "learning_rate": {"type": "float", "low": 1e-5, "high": 1e-3, "log": True},
    "warmup_ratio": {"type": "float", "low": 0.005, "high": 0.01},
    "weight_decay": {"type": "float", "low": 0.01, "high": 0.1},
    "dropout_rate": {"type": "float", "low": 0.0, "high": 0.7},
    "lr_scheduler_type": {"type": "categorical", "choices": ["cosine"]},
    "task_weights": {"type": "float", "low": 0.1, "high": 2.0}
}

In the code above, we've defined `task_columns` as `["cell_type", "disease_state"]`. This means our model will be trained to classify cells based on two tasks:
1. Identifying the cell type
2. Determining the disease state
3. Note: "unique_cell_id" is a required column in the dataset for logging and inference purposes

These column names should correspond to actual columns in your dataset. Each column should contain the labels for that specific classification task.

For example, your dataset might look something like this:

    | unique_cell_id | input_ids | ... | cell_type | disease_state |
    |----------------|-----------|-----|-----------|---------------|
    | cell1          | ...       | ... | neuron    | healthy       |
    | cell2          | ...       | ... | astrocyte | diseased      |
    | ...            | ...       | ... | ...       | ...           |
The model will learn to predict classes within 'cell_type' and 'disease_state' 

## 3. Initialize the MTLClassifier

Now, let's create an instance of the MTLClassifier with our defined parameters and task columns.

In [ ]:
mc = MTLClassifier(
    task_columns=task_columns,  # Our defined classification tasks
    study_name="MTLClassifier_example",
    pretrained_path=pretrained_path,
    train_path=train_path,
    val_path=val_path,
    test_path=test_path,
    model_save_path=model_save_path,
    results_dir=results_dir,
    tensorboard_log_dir=tensorboard_log_dir,
    hyperparameters=hyperparameters,
    n_trials=15,  # Number of trials for hyperparameter optimization (at least 50 suggested)
    epochs=1,    # Number of training epochs (1 suggested to prevent overfitting)
    batch_size=8, # Adjust based on available GPU memory
    seed=42
)

## 4. Run Hyperparameter Optimization

Now, let's run the Optuna study to optimize our hyperparameters for both classification tasks.

In [ ]:
mc.run_optuna_study()

## 5. Evaluate the Model on Test Data

After optimization, we can evaluate our model on the test dataset. This will provide performance metrics for both classification tasks. CSV containing following keys will be generated in specified results directiory "Cell ID, task(1...n) True,task(1.,.n) Pred,task(1...n) Probabilities"

In [ ]:
mc.load_and_evaluate_test_model()

## 6. (Optional) Manual Hyperparameter Tuning

If you prefer to set hyperparameters manually, you can use the following approach:

In [ ]:
manual_hyperparameters = {
    "learning_rate": 0.001,
    "warmup_ratio": 0.01,
    "weight_decay": 0.1,
    "dropout_rate": 0.1,
    "lr_scheduler_type": "cosine",
    "task_weights": [1, 1],  # Weights for each task (cell_type, disease_state)
    "max_layers_to_freeze": 2
}

mc_manual = MTLClassifier(
    task_columns=task_columns,
    study_name="mtl_manual",
    pretrained_path=pretrained_path,
    train_path=train_path,
    val_path=val_path,
    test_path=test_path,
    model_save_path=model_save_path,
    results_dir=results_dir,
    tensorboard_log_dir=tensorboard_log_dir,
    manual_hyperparameters=manual_hyperparameters,
    use_manual_hyperparameters=True,
    epochs=10,
    batch_size=32,
    seed=42
)

mc_manual.run_manual_tuning()

# Geneformer In Silico Perturber Tutorial (MTL Quantized)
This demonstrates how to use the Geneformer In Silico Perturber with a Multi-Task Learning (MTL) model in a quantized configuration to optimize runtime and memory.

In [ ]:
from geneformer import InSilicoPerturber, EmbExtractor, InSilicoPerturberStats

In [ ]:
# Define paths
model_directory = "/path/to/model/save/path"
input_data_file = "/path/to/input/data.dataset"
output_directory = "/path/to/output/directory"
output_prefix = "mtl_quantized_perturbation"

# Define parameters
perturb_type = "delete"  # or "overexpress"

# Define cell states to model
cell_states_to_model = {
    "state_key": "disease_state", 
    "start_state": "disease", 
    "goal_state": "control"
}

# Define filter data
filter_data_dict = {
    "cell_type": ["Fibroblast"]
}

## 3. Extract State Embeddings

Before we initialize the InSilicoPerturber, we need to extract the state embeddings using the EmbExtractor.

In [ ]:
# Initialize EmbExtractor
embex = EmbExtractor(
    filter_data_dict=filter_data_dict,
    max_ncells=1000, # Number of cells to extract embeddings for
    emb_layer=0,  # Use the second to last layer
    emb_mode = "cls",  # Use CLS token embedding for V2 model
    summary_stat="exact_mean",
    forward_batch_size=8, # Adjust based on available GPU memory
    nproc=4
)

# Extract state embeddings
state_embs_dict = embex.get_state_embs(
    cell_states_to_model,
    model_directory=model_directory,
    input_data_file=input_data_file,
    output_directory=output_directory,
    output_prefix=output_prefix
)

## 4. Initialize the InSilicoPerturber

Now that we have our state embeddings, let's create an instance of the InSilicoPerturber with MTL and quantized configurations.

In [ ]:
# Initialize InSilicoPerturber
isp = InSilicoPerturber(
    perturb_type=perturb_type,
    genes_to_perturb="all",  # Perturb all genes
    model_type="MTLCellClassifier-Quantized",  # Use quantized MTL model
    emb_mode="cls",  # Use CLS token embedding for V2 model
    cell_states_to_model=cell_states_to_model,
    state_embs_dict=state_embs_dict,
    max_ncells=1000,  # Number of cells to perturb (larger number increases power)
    emb_layer=0,  
    forward_batch_size=8, # Adjust based on available GPU memory
    nproc=1
)

## 5. Run In Silico Perturbation

Run the in silico perturbation on the dataset.

In [ ]:
# Run perturbation and output intermediate files
isp.perturb_data(
    model_directory=model_directory,
    input_data_file=input_data_file,
    output_directory=output_directory,
    output_prefix=output_prefix
)

## 6. Process Results with InSilicoPerturberStats

After running the perturbation, we'll use InSilicoPerturberStats to process the intermediate files and generate the final statistics.

In [ ]:
# Initialize InSilicoPerturberStats
ispstats = InSilicoPerturberStats(
    mode="goal_state_shift",
    genes_perturbed="all",
    combos=0,
    anchor_gene=None,
    cell_states_to_model=cell_states_to_model
)

# Process stats and output final .csv
ispstats.get_stats(
    input_data_file,
    None,
    output_directory,
    output_prefix
)